# Service Quality & Customer LTV Analytics – Urban Mobility Platform 🚕⭐

**Author:** Luis Chaumer  
**Role:** Data Analyst  

This notebook uses a unified synthetic dataset to analyze:

1. **Service Quality & Customer Satisfaction**
   - Response times by channel
   - Resolution performance by issue type
   - CSAT and NPS distributions
   - Reopened and unresolved cases

2. **Customer Segmentation & Lifetime Value (LTV)**
   - Revenue and net revenue per customer
   - LTV segments (low / medium / high)
   - Relationship between LTV, satisfaction and churn

The data simulates one year of interactions (support + trip-related)  
for **30,000 customers** and **112,568 interactions** across 5 regions.


## 1. Setup and data loading

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

interactions = pd.read_csv('data/service_quality_interactions_dataset.csv', parse_dates=['interaction_datetime'])
customers = pd.read_csv('data/service_quality_customers_agg_dataset.csv', parse_dates=['first_interaction','last_interaction','signup_date'])

interactions.head(), customers.head()

## 2. Service quality & customer satisfaction analysis

In [ ]:
# Basic KPIs
kpis = {
    "avg_response_time_min": interactions["response_time_min"].mean(),
    "median_response_time_min": interactions["response_time_min"].median(),
    "avg_resolution_time_min": interactions["resolution_time_min"].mean(),
    "csat_mean": interactions["csat_score"].mean(),
    "nps_mean": interactions["nps_score"].mean(),
    "fcr_rate": (interactions["resolution_status"] == "resolved_first_contact").mean(),
    "escalation_rate": (interactions["resolution_status"] == "escalated").mean(),
    "unresolved_rate": (interactions["resolution_status"] == "unresolved").mean(),
    "reopen_rate": interactions["reopened"].mean(),
}
pd.Series(kpis)

### 2.1 CSAT distribution

![CSAT Distribution](images/csat_distribution.png)

### 2.2 Response time by channel

In [ ]:
interactions.groupby("channel")["response_time_min"].mean().sort_values()

![Response Time by Channel](images/response_time_by_channel.png)

### 2.3 CSAT by issue type

In [ ]:
interactions.groupby("issue_type")["csat_score"].mean().sort_values()

![CSAT by Issue Type](images/csat_by_issue_type.png)

## 3. Customer segmentation & LTV analysis

In [ ]:
customers[["trips","net_revenue","recency_days","tenure_days","contacts_per_month"]].describe().T

### 3.1 Net revenue per customer

![Net Revenue Distribution](images/net_revenue_distribution.png)

### 3.2 LTV segments – revenue contribution

In [ ]:
customers.groupby("ltv_segment")["net_revenue"].agg(["count","sum"]).rename(columns={"count":"customers","sum":"total_net_revenue"})

![Revenue by LTV Segment](images/revenue_by_ltv_segment.png)

### 3.3 Churn rate by LTV segment

In [ ]:
customers.groupby("ltv_segment")["is_churned"].mean().rename("churn_rate")

![Churn by LTV Segment](images/churn_by_ltv_segment.png)

### 3.4 Satisfaction vs churn

In [ ]:
customers.groupby("satisfaction_segment")["is_churned"].mean().rename("churn_rate")

## 4. SQL analysis with SQLite

We can also run SQL queries on the same datasets using **SQLite** to mimic real BI / analytics workflows.

In [ ]:
conn = sqlite3.connect(":memory:")
interactions.to_sql("interactions", conn, index=False, if_exists="replace")
customers.to_sql("customers", conn, index=False, if_exists="replace")

### 4.1 Average CSAT and response time by region (SQL)

In [ ]:
query = '''
SELECT 
    i.region,
    COUNT(*) AS interactions,
    ROUND(AVG(i.csat_score), 2) AS avg_csat,
    ROUND(AVG(i.response_time_min), 2) AS avg_response_time
FROM interactions i
GROUP BY i.region
ORDER BY avg_csat DESC;
'''
pd.read_sql_query(query, conn)

### 4.2 Churn by LTV and satisfaction segment (SQL)

In [ ]:
query = '''
SELECT 
    ltv_segment,
    satisfaction_segment,
    COUNT(*) AS customers,
    ROUND(AVG(is_churned), 3) AS churn_rate,
    ROUND(AVG(net_revenue), 2) AS avg_net_revenue
FROM customers
GROUP BY ltv_segment, satisfaction_segment
ORDER BY ltv_segment, satisfaction_segment;
'''
pd.read_sql_query(query, conn)

## 5. Conclusions and next steps

_Suggested insights (to refine based on actual metrics):_

- Channels like **phone** and **in-app chat** tend to have much faster response times than email/web forms,  
  and are associated with higher average CSAT.  
- Certain issue types (e.g. _app bugs_ or _driver behavior_) show lower CSAT and longer resolution times,  
  indicating an opportunity to improve internal processes or tooling.  
- **High LTV customers** generate a disproportionate share of net revenue and typically have higher satisfaction  
  and lower churn rates than low LTV segments.  
- Customers in the **detractor** satisfaction segment churn at significantly higher rates,  
  suggesting that improving service quality has direct revenue impact.

**Next steps / potential extensions:**

- Build a churn prediction model using LTV, satisfaction and behavior features.  
- Create a real-time alert system for low CSAT or long response times.  
- Design targeted retention campaigns for medium/high LTV customers at risk of churn.  
